# Install libraries

In [1]:
#!pip install pandas tqdm scikit-learn -q

# Imports

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

# Creating directory for storing the CSVs
directory = 'Processed_CSVs'
if(not os.path.exists(directory)):
    os.makedirs(directory)
    print("Directory created successfully")

# Read CSV
Also, clean the CSV by deleting useless columns and rows which contain non-alphabetical characters

In [2]:
df = pd.read_csv('all_cars_1941-2022.csv')

In [3]:
# A bit of preprocessing and data cleaning
df.drop(['city', 'mixed', 'hwy'], axis=1, inplace=True)

# Define the pattern
pattern = r'[+!#]'
columns_to_check = ['trim', 'body', 'engine_position', 'engine_type',
                    'engine_compression', 'fuel', 'drive', 'transmission']

rows_to_drop = df[columns_to_check].apply(lambda col: col.str.contains(pattern)).any(axis=1)

# Drop rows containing non-alphabetical characters
df = df[~rows_to_drop]

In [4]:
# Replacing values
df['body'] = df['body'].replace({'Sedan ':'Sedan', 'Convertible ':'Convertible'})
df['cylinder'] = df['cylinder'].replace(0., np.nan)
df['engine_type'] = df['engine_type'].replace({'in-line': 'Inline', 'V ': 'V'})
df['engine_position'] = df['engine_position'].replace({'Front ': 'Front', 'rear': 'Rear'})
df['fuel'] = df['fuel'].replace({'Gasoline ': 'Gasoline', 'Premium Unleaded (Recommended)': 'Premium Unleaded',\
                                'Premium Unleaded (Required)': 'Premium Unleaded', 'Flex-Fuel (Unleaded/E85)': 'FlexFuel',\
                                'Flex-Fuel (Unleaded/Natural Gas)': 'FlexFuel', 'Flex-Fuel (Premium Unleaded Requ': 'FlexFuel',\
                                'Flex-Fuel (Premium Unleaded Reco': 'FlexFuel', 'Flex-Fuel (Premium Unleaded/E85)': 'FlexFuel',\
                                'electric': 'Electric',})
df['drive'] = df['drive'].replace({'All Wheel Drive': 'AWD', 'Four Wheel Drive': '4WD', 'Front Wheel Drive': 'FWD', 'Rear Wheel Drive': 'RWD',\
                                'AWD ': 'AWD', 'front': 'FWD','Front': 'FWD',})
df['transmission'] = df['transmission'].replace({'CVT ': 'CVT', '4-speed automatic ':'4-speed automatic', '6-speed manual ':'6-speed manual',\
                                '5-speed automatic ':'5-speed automatic', 'automatic ':'Automatic', 'manual':'Manual','6-speed automatic ':'6-speed automatic',\
                                '7-speed automatic ':'7-speed automatic', '8-speed automatic ':'8-speed automatic',\
                                                        })
df['doors'] = df['doors'].replace(0., np.nan)
df['fuel_cap_l'] = df['fuel_cap_l'].replace(0., np.nan)

In [5]:
df.head(5)

,make,country,model,trim,year,body,engine_position,engine_cc,cylinder,engine_type,...,power_rpm,torque_nm,torque_rpm,engine_compression,fuel,drive,transmission,doors,fuel_cap_l,sold_in_us
0,Mercedes-Benz,Germany,250,2.8,1970,NaN,Front,2776,6.0,Inline,...,5000.0,216.0,3200.0,8.7:1,NaN,RWD,Manual,4.0,78.0,1
1,Mercedes-Benz,Germany,250,2.8 Automatic,1970,NaN,Front,2778,6.0,Inline,...,5000.0,217.0,3200.0,NaN,Gasoline,RWD,Automatic,4.0,65.0,1
2,Mercedes-Benz,Germany,280,SL,1970,NaN,Front,2778,6.0,Inline,...,NaN,258.0,NaN,9.5:1,NaN,RWD,Manual,NaN,NaN,1
3,Mercedes-Benz,Germany,600,Pullman,1970,NaN,Front,6330,8.0,Inline,...,4000.0,500.0,2800.0,NaN,Gasoline,RWD,NaN,NaN,NaN,1
4,Mercedes-Benz,Germany,C 111,NaN,1970,NaN,Middle,2400,NaN,NaN,...,7000.0,392.0,4000.0,9.3:1,NaN,RWD,Manual,2.0,120.0,1


Count NaNs values

In [6]:
# Count how many NaN values are in each column
nan_counts = df.isna().sum()
print(nan_counts)

# Dataframe shape
print(f"DF Shape: {df.shape}")

make                       0
country                    0
model                      0
trim                   13090
year                       0
body                   15664
engine_position         1236
engine_cc                  0
cylinder                1521
engine_type             1263
valves_per_cylinder    17618
power_ps                4195
power_rpm              27872
torque_nm               6440
torque_rpm             30136
engine_compression     31247
fuel                    8419
drive                   2069
transmission            9592
doors                  10541
fuel_cap_l             20455
sold_in_us                 0
dtype: int64
DF Shape: (79212, 22)


# Option 0: Save Database with simply few touches
In this case, we just removed 100 rows which were noisy (non-alphabetical number and columns which we do not need)

In [28]:
# Export the Dataframe to a CSV file separated by a comma
df.to_csv(os.path.join(directory,'all_cars_1941-2022_cleaned.csv'), index=False)

# Option 1: Drop NaN values

In [35]:
df = df.dropna()

# Count how many NaN values are in each column
nan_counts = df.isna().sum()
print(nan_counts)

# Dataframe shape
print(f"DF Shape: {df.shape}")

make                   0
country                0
model                  0
trim                   0
year                   0
body                   0
engine_position        0
engine_cc              0
cylinder               0
engine_type            0
valves_per_cylinder    0
power_ps               0
power_rpm              0
torque_nm              0
torque_rpm             0
engine_compression     0
fuel                   0
drive                  0
transmission           0
doors                  0
fuel_cap_l             0
sold_in_us             0
dtype: int64
DF Shape: (13145, 22)


## Export the Dataset

In [36]:
# Export the Dataframe to a CSV file separated by a comma
df.to_csv(os.path.join(directory,'all_cars_1941-2022_nans_dropped.csv'), index=False)

# Option 2: Use Sklearn to fill NaN values

In [7]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder

## String columns
We first populate the string columns as they require a different approach.
The scheme is the following:
- 1. Selects columns that are strings and do not contain NaN values, then temporarily drops these columns from the DataFrame.
- 2. Use _`LabelEncoder`_ to each remaining string column, store the mappings in a dictionary (encoder_mappings), and encodes the string values as numerical values (as required by the Classifier).
- 3. Train a Classifier (in this case, `KNeighborsClassifier`) to predict missing values in each string column. Along with predicting the values, we evaluate the performance of the classifier on a small portion of the CSV.
- 4. Map the predicted numerical values back to their corresponding strings using the encoder_mappings dictionary created before.
- 5. Replace the original DataFrame's string columns with the predicted values from the Classifier (removing NaNs)

In [8]:
# STEP 1: Convert all the string values to numerical values

# Get the list of columns which are strings and CONTAIN NaN values.
string_columns = df.select_dtypes(include=['object']).columns[df.select_dtypes(include=['object']).isna().sum() == 0]
df_filtered = df.drop(columns=string_columns, inplace=False) # Drop these columns

# Select only the columns which are strings now and replace NaN values with 'NaN'
df_filtered = df_filtered.select_dtypes(include=['object'])
df_filtered[pd.isnull(df_filtered)]  = 'NaN'

In [9]:
# STEP 2: Apply LabelEncoder to each of these columns and store the mappings

encoder_mappings = {}

# Apply LabelEncoder to each of these columns and store the mappings
for col in df_filtered.columns:
    label_encoder = LabelEncoder()

    # LabelEncoder the column
    column = label_encoder.fit_transform(df_filtered[col])
    df_filtered[col] = column
    encoder_mappings[col] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [10]:
# STEP 3: Train a Classifier (e.g., KNeighborsClassifier)
classifier = KNeighborsClassifier()
test_preds = []

# Iterate over columns with missing values
for col in tqdm(df_filtered.columns, desc='Fitting classifier to columns with missing values'):
    # Continue until all NaNs in the column are filled. We must take the NaN value from the encoder_mappings dictionary
    nan_value = encoder_mappings[col]['NaN']
    # Continue until all NaNs in the column are filled. It means that we need to check if in "col" there is still value 12035.
    while nan_value in df_filtered[col].values:
        # Extract target variable (column with missing values) and features (rest of the columns)
        target = df_filtered[col]
        features = df_filtered.drop(columns=col)

        # Split the data into two parts: one with non-null values and one with null values for the target column
        # In order to check this, we need to place a mask based on the nan_value
        mask = target != nan_value
        train_data = features[mask]
        test_data = features[~mask]
        train_target = target[mask]

        # Get the last 10% of the training data for validation
        split_index = int(len(train_data) * 0.9)
        train_data, val_data = train_data.iloc[:split_index], train_data.iloc[split_index:]
        train_target, val_target = train_target.iloc[:split_index], train_target.iloc[split_index:]

        # Fit the regressor
        classifier.fit(train_data, train_target)

        # Test the regressor
        test_preds.append(classifier.score(val_data, val_target))

        # Predict missing values
        predictions = classifier.predict(test_data)

        for i in range(len(predictions)):
            predictions[i] = int(predictions[i])

        # Update DataFrame with predicted values. We need to place the predictions in the original DataFrame
        # It means that we need to take all the indexes from the original DataFrame and place the predictions in the right place
        # This can be done through the mask and the indexes of the test_data
        df_filtered.loc[test_data.index, col] = predictions

# Check the score of the classifier
print(f"Classifier score: {np.mean(test_preds)}")

Fitting classifier to columns with missing values: 100%|██████████| 8/8 [00:06<00:00,  1.28it/s]

Classifier score: 0.8008686568867072


In [11]:
# STEP 4: Map back the numerical values into strings
# STEP 5: Replace the NaN values with the predicted values
for col in tqdm(df_filtered.columns, desc='Mapping back numerical values into strings'):
    df[col] = df_filtered[col].apply(lambda x: next((key for key, value in encoder_mappings[col].items() if value == x), None))  

# Count how many NaN values are in each column
nan_counts = df.isna().sum()
print(nan_counts) # We can see from this print that the string columns now do not contain NaN values.

# Dataframe shape
print(f"DF Shape: {df.shape}") # The shape of the DataFrame is the same as the original one

Mapping back numerical values into strings: 100%|██████████| 8/8 [00:31<00:00,  3.92s/it]

make                       0
country                    0
model                      0
trim                       0
year                       0
body                       0
engine_position            0
engine_cc                  0
cylinder                1521
engine_type                0
valves_per_cylinder    17618
power_ps                4195
power_rpm              27872
torque_nm               6440
torque_rpm             30136
engine_compression         0
fuel                       0
drive                      0
transmission               0
doors                  10541
fuel_cap_l             20455
sold_in_us                 0
dtype: int64
DF Shape: (79212, 22)


## Numerical columns
Once the string columns have been populated, we need to populate columns which contain numerical values

In [12]:
# Get the list of columns which are strings and don't contain NaN values
numerical_columns = df.select_dtypes(exclude=['object']).columns[df.select_dtypes(exclude=['object']).isna().sum() == 0]

# Drop the columns which contain numerical values and not NaN values (We don't want to predict these values, only NaNs)
df_numerical_filtered = df.drop(columns=numerical_columns, inplace=False)

# Select only the columns which are numbers now and replace NaN values with a proper 'NaN'
df_numerical_filtered = df_numerical_filtered.select_dtypes(exclude=['object'])
df_numerical_filtered[pd.isnull(df_numerical_filtered)]  = -1

numerical_columns = df_numerical_filtered.columns

for col in numerical_columns:
    df[col] = df_numerical_filtered[col]

In [13]:
# Use a temporary DataFrame to store the original DataFrame, because we need to convert the string columns to numerical values
# once again. Work on the copy, then pass the predicted values to the original DataFrame.
df_copy = df.copy()

# Apply LabelEncoder to each of the string columns
string_columns = df_copy.select_dtypes(include=['object'])

for col in string_columns:
    label_encoder = LabelEncoder()

    # LabelEncoder the column
    column = label_encoder.fit_transform(df_copy[col])
    df_copy[col] = column

In [14]:
# Now we need to enter in each column (taking them from df_numerical_filtered) and predict the NaN values based on all the other columns
# We will use the same approach as before, but now we will use a regressor instead of a classifier
regressor = KNeighborsRegressor()
nan_value = -1

# Iterate over columns with missing values
for col in tqdm(numerical_columns, desc='Fitting classifier to columns with missing values'):
    while nan_value in df_copy[col].values:
        # Extract target variable (column with missing values) and features (rest of the columns)
        target = df_copy[col]
        features = df_copy.drop(columns=col)

        # Split the data into two parts: one with non-null values and one with null values for the target column
        # In order to check this, we need to place a mask based on the nan_value
        mask = target != nan_value
        train_data = features[mask]
        test_data = features[~mask]
        train_target = target[mask]

        # Get the last 10% of the training data for validation
        split_index = int(len(train_data) * 0.9)
        train_data, val_data = train_data.iloc[:split_index], train_data.iloc[split_index:]
        train_target, val_target = train_target.iloc[:split_index], train_target.iloc[split_index:]

        # Fit the regressor
        regressor.fit(train_data, train_target)

        # Test the regressor
        test_preds.append(regressor.score(val_data, val_target))

        # Predict missing values
        predictions = regressor.predict(test_data)

        for i in range(len(predictions)):
            predictions[i] = int(predictions[i])

        # Update DataFrame with predicted values. We need to place the predictions in the original DataFrame
        # It means that we need to take all the indexes from the original DataFrame and place the predictions in the right place
        # This can be done through the mask and the indexes of the test_data
        df_copy.loc[test_data.index, col] = predictions

# Check the score of the regressor
print(f"Regressor score: {np.mean(test_preds)}")

Fitting classifier to columns with missing values: 100%|██████████| 8/8 [00:06<00:00,  1.16it/s]

Regressor score: 0.8119647311368009


In [15]:
# Map back the numerical values into the original dataframe
for col in numerical_columns:
    df[col] = df_copy[col]

# Count how many NaN values are in each column
nan_counts = df.isna().sum()
print(nan_counts)

# Dataframe shape
print(f"DF Shape: {df.shape}")

make                   0
country                0
model                  0
trim                   0
year                   0
body                   0
engine_position        0
engine_cc              0
cylinder               0
engine_type            0
valves_per_cylinder    0
power_ps               0
power_rpm              0
torque_nm              0
torque_rpm             0
engine_compression     0
fuel                   0
drive                  0
transmission           0
doors                  0
fuel_cap_l             0
sold_in_us             0
dtype: int64
DF Shape: (79212, 22)


## Export the Dataset

In [16]:
# Export the Dataframe to a CSV file separated by a comma
df.to_csv(os.path.join(directory,'all_cars_1941-2022_sklearn.csv'), index=False)